In [1]:
# Cardiac EXPERT EVAL - Evals PubMedQA

Model was trained on blood_heart_circulation_qa training data. 17 epochs /160 steps

In [2]:
import os
import json
import datetime
from tqdm import tqdm
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)

In [3]:
# Change paths as needed
model_path = "moeme/model/cardiovascular_expert_model/"

#Test data is pubmedqa
data_path = "moeme/model/model_eval/pubmedqa/data/ori_pqal.json"
ground_truth = "moeme/model/model_eval/pubmedqa/data/test_ground_truth.json"

# Save predictions
output_file = "expert1_predictions.json"

limit = None  # e.g., 100 for testing subset
verbose = False  # or False

#Enable bfloat16 or 4-bit if supported by GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Device set to use cuda:0


In [4]:
def pubmed_prompt(model_pipeline, pmid, data_file, verbose=False):
    intro = "TASK: Your task is to answer biomedical questions using the given abstract. Only output yes, no, or maybe as answer."
    
    contexts = data_file[pmid].get("CONTEXTS", "")
    if isinstance(contexts, list):
        contexts = " ".join(contexts)
    
    question = data_file[pmid]["QUESTION"]
    full_prompt = f"{intro} INPUT: {contexts} {question} OUTPUT:"
    
    if verbose:
        print(f"\nPrompt: {full_prompt}")
    
    output = model_pipeline(full_prompt, max_new_tokens=20)
    full_response = output[0]['generated_text']
    answer_part = full_response[len(full_prompt):].strip().lower()
    
    for ans in ["yes", "no", "maybe"]:
        if ans in answer_part[:20]:
            prediction = ans
            break
    else:
        counts = {ans: answer_part.count(ans) for ans in ["yes", "no", "maybe"]}
        prediction = max(counts, key=counts.get)

    if verbose:
        print(f"Model Answer: {prediction} | True Answer: {data_file[pmid]['final_decision']}")
    
    return prediction


In [5]:
# Load PubMedQA JSON
with open(data_path, 'r') as f:
    pubmedqa_data = json.load(f)

with open(ground_truth, 'r') as f:
    gt_data = json.load(f)

# Backup output file if it exists
if os.path.exists(output_file):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    os.rename(output_file, f"{os.path.splitext(output_file)[0]}_{timestamp}.json")

# Limit dataset for test runs
pmids = list(gt_data.keys())
if limit:
    pmids = pmids[:limit]

# Run predictions
results = {}
correct = 0
total = 0

for pmid in tqdm(pmids, desc="Evaluating PubMedQA"):
    prediction = pubmed_prompt(pipe, pmid, pubmedqa_data, verbose=verbose)
    results[pmid] = prediction

    # Save progress
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

    if prediction == pubmedqa_data[pmid]["final_decision"]:
        correct += 1
    total += 1
    acc = correct / total
    tqdm.write(f"[{total}] Accuracy: {acc:.4f}")


Evaluating PubMedQA:   0%|          | 1/500 [00:01<14:50,  1.78s/it]

[1] Accuracy: 1.0000


Evaluating PubMedQA:   0%|          | 2/500 [00:03<13:24,  1.62s/it]

[2] Accuracy: 1.0000


Evaluating PubMedQA:   1%|          | 3/500 [00:04<12:56,  1.56s/it]

...

[498] Accuracy: 0.5442


Evaluating PubMedQA: 100%|█████████▉| 499/500 [12:33<00:01,  1.52s/it]

[499] Accuracy: 0.5431


Evaluating PubMedQA: 100%|██████████| 500/500 [12:34<00:00,  1.51s/it]

[500] Accuracy: 0.5420


In [6]:
# Evaluation.py - Original PubMed Scorer

import json
from sklearn.metrics import accuracy_score, f1_score

#load results
with open(output_file, 'r') as f:
    predictions = json.load(f)


pmids = list(gt_data)
truth = [gt_data[pmid] for pmid in pmids]
preds = [predictions[pmid] for pmid in pmids]

acc = accuracy_score(truth, preds)
maf = f1_score(truth, preds, average='macro')

print('Accuracy %f' % acc)
print('Macro-F1 %f' % maf)


Accuracy 0.542000
Macro-F1 0.260790
